<h2 style="color:rgb(37, 131, 207);">Fractured or not-fractured that is the question</h2>

This project uses **computer vision** to help identify fractured-bones in X-ray images. It's a binary classification problem using deep learing.

<h3 style="color:rgb(37, 131, 207);">Objective</h3>

- To acquire X-ray images of fractured and healthy bones to train the CNN.
- Split the dataset into train, test and validation.
- Train a simple CNN.
- Predict the outcome.
- Calculate the accuracy.
- Fine tune the model.

<h3 style="color:rgb(37, 131, 207);">Import all the libraries here</h3>

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from PIL import Image
import time

<h3 style="color:rgb(37, 131, 207);">Check Tensorflow compilation</h3>

In [ ]:
print("TensorFlow version:", tf.__version__)
print("Is GPU available:", tf.config.list_physical_devices('GPU'))

In [ ]:
from tensorflow.python.platform import build_info as tf_build_info
print("CUDA support:", tf_build_info.build_info.get("cuda_version", "Not built with CUDA"))
print("cuDNN support:", tf_build_info.build_info.get("cudnn_version", "Not built with cuDNN"))

In [ ]:
print("Compiled with CUDA:", tf.sysconfig.get_build_info()["cuda_version"])

- Tensorflow is compiled with CUDA. So it has GPU support and CNN training would be fast.

<h3 style="color:rgb(37, 131, 207);">Data extraction</h3>

In [ ]:
data_path = os.getcwd().replace("code", "data")

In [ ]:
# Verify the dataset path and structure
print("Dataset path:", data_path)
print("Contents of dataset_path:", os.listdir(data_path))

In [ ]:
train_dir = os.path.join(data_path, "train")
test_dir = os.path.join(data_path, "test")
validation_dir = os.path.join(data_path, "val")

In [ ]:
# Verify train, test and validation directories
print("Train directory contents:", os.listdir(train_dir))
print("Test directory contents:", os.listdir(test_dir))
print("Validation directory contents:", os.listdir(validation_dir))

<h4 style="color:rgb(37, 131, 207);">Remove spaces and rename files</h4>

In [ ]:
def rename_files_recursive(folder_path):
    """
    Recursively renames files in the given folder and its subfolders,
    replacing spaces with underscores in filenames.
    """
    for dirpath, _, filenames in os.walk(folder_path):
        for filename in filenames:
            if ' ' in filename:
                old_path = os.path.join(dirpath, filename)
                new_filename = filename.replace(' ', '_')
                new_path = os.path.join(dirpath, new_filename)
                os.rename(old_path, new_path)
                print(f'Renamed: "{old_path}" → "{new_path}"')

rename_files_recursive(os.path.join(train_dir, "fractured"))
rename_files_recursive(os.path.join(train_dir, "not fractured"))
rename_files_recursive(os.path.join(test_dir, "fractured"))
rename_files_recursive(os.path.join(test_dir, "not fractured"))
rename_files_recursive(os.path.join(validation_dir, "fractured"))
rename_files_recursive(os.path.join(validation_dir, "not fractured"))

<h4 style="color:rgb(37, 131, 207);">Remove corrupted images using tf.io.read_file()</h4>

In [ ]:
def remove_invalid_images(folder_path):
    removed = 0
    for root, _, files in os.walk(folder_path):
        for fname in files:
            fpath = os.path.join(root, fname)
            try:
                img_read = tf.io.read_file(fpath)
                img_2 = tf.image.decode_image(img_read)
            
            except tf.errors.InvalidArgumentError:
                print(f"Removing: {fpath}")
                os.remove(fpath)
                removed += 1
    print(f"Removed {removed} corrupted/invalid images from {folder_path}")

# Check all your datasets
remove_invalid_images(os.path.join(train_dir, "fractured"))
remove_invalid_images(os.path.join(train_dir, "not fractured"))
remove_invalid_images(os.path.join(test_dir, "fractured"))
remove_invalid_images(os.path.join(test_dir, "not fractured"))
remove_invalid_images(os.path.join(validation_dir, "fractured"))
remove_invalid_images(os.path.join(validation_dir, "not fractured"))

<h3 style="color:rgb(37, 131, 207);">Creating TensorFlow data object & visualizing the dataset</h3>

In [ ]:
# batch size
batch_size = 32

# image_size
img_size = (160, 160)

<h3 style="color:rgb(37, 131, 207);">Train-test split</h3>

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir, shuffle=True, batch_size=batch_size, image_size=img_size, labels='inferred')
test_dataset = tf.keras.utils.image_dataset_from_directory(test_dir, shuffle=True, batch_size=batch_size, image_size=img_size, labels='inferred')
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir, shuffle=True, batch_size=batch_size, image_size=img_size, labels='inferred')

In [ ]:
class_names = train_dataset.class_names

In [ ]:
class_names

In [ ]:
# sampling images from training dataset
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(2):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.suptitle("Sample Images from Training Dataset", fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
# sampling images from testing dataset
plt.figure(figsize=(10, 10))
for images, labels in test_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.suptitle("Sample Images from Testing Dataset", fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
# sampling images from validation dataset
plt.figure(figsize=(10, 10))
for images, labels in validation_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.suptitle("Sample Images from Validation Dataset", fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
batch_size

<h3 style="color:rgb(37, 131, 207);">Prefetching for optimization</h3>

In [ ]:
tf.data.AUTOTUNE

In [ ]:
autotune = tf.data.AUTOTUNE

In [ ]:
# Optimize dataset performance with prefetching
train_dataset = train_dataset.prefetch(buffer_size=autotune)
validation_dataset = validation_dataset.prefetch(buffer_size=autotune)
test_dataset = test_dataset.prefetch(buffer_size=autotune)

In [ ]:
len(train_dataset)

In [ ]:
len(test_dataset)

In [ ]:
len(validation_dataset)

<h3 style="color:rgb(37, 131, 207);">Data Augmentation</h3>

In [ ]:
# Data augmentation
horizontal_flips = tf.keras.layers.RandomFlip('horizontal')
radians = tf.keras.layers.RandomRotation(0.02)

In [ ]:
data_augmentation = tf.keras.Sequential([
    horizontal_flips,
    radians
])

In [ ]:
type(data_augmentation)

In [ ]:
# Visualize augmented images
plt.figure(figsize=(12, 12))
for images, _ in train_dataset.take(1):
    sample_image = images[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(sample_image, 0))
        plt.imshow(augmented_image[0].numpy().astype("uint8"))
        plt.axis('off')
        plt.title(f"Augmented Image {i+1}", fontsize=12)
plt.suptitle("Sample Augmented Images", fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()

<h3 style="color:rgb(37, 131, 207);">Transfer learning with MobileNetV2</h3>

In [ ]:
img_shape = img_size + (3,) # MobileNetV2 only works with RGB

In [ ]:
img_size

In [ ]:
img_shape

<h3 style="color:rgb(37, 131, 207);">CNN Architechture</h3>

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape, 
                                               include_top=False, # Exclude the ImageNet classifier at the top
                                               weights='imagenet') # Use the pre-trained weights from ImageNet

In [ ]:
base_model.trainable = False # Freeze the base model to keep the pre-trained weights

In [ ]:
inputs = tf.keras.Input(shape=img_shape)
x = data_augmentation(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = base_model(x, training=False)

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D() # layer object

# Apply the layer to MobileNetV2
x = global_average_layer(x)
# x = tf.keras.layers.Dropout(0.2)(x) ### Optional: Add a dropout layer

prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid') # layer object
# Apply Fully Connected Layer to predict the class
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
print("Number of neurons:", prediction_layer.units)

In [ ]:
model.layers

In [ ]:
model.summary()

<h3 style="color:rgb(37, 131, 207);">Training the model</h3>

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=batch_size, # Learning rate will decrease every batch
    decay_rate=0.8 # rate of decrease means 90% of the learning rate is decreased
)

In [ ]:
# Compile the model structure for training
base_learning_rate = 0.1
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), # Adam deals with gradient descent
              loss=tf.keras.losses.BinaryCrossentropy(), # measures the difference between two probability distributions
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5, name='accuracy')]) # calculates how often predictions match binary labels

<h3 style="color:rgb(37, 131, 207);">Early stop, epochs & saving the best model</h3>

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(filepath='best_model.keras', monitor='val_loss', save_best_only=True)

In [ ]:
# Epochs are the number of times the model will see the entire dataset
# Start with a small number of epochs to prevent overfitting
# Epochs usually are in the range of 10 to 100. Standard is up to 50.
initial_epochs = 10

<h3 style="color:rgb(37, 131, 207);">Training & saving (best model + metrics)</h3>

In [ ]:
# start time
start = time.time()

import json

# Train the model with callbacks
history = model.fit(
    train_dataset,
    epochs=initial_epochs,
    validation_data=validation_dataset,
    callbacks=[early_stopping, model_checkpoint], # Callbacks are used to customize the training process
    verbose=2  # Verbosity level: 1 = progress bar, 2 = one line per epoch
)

# Save training history
with open('training_history.json', 'w') as file: # open creates a file
    json.dump(history.history, file) # dump saves it as a json file

# end time
end = time.time()
print(f"Elapsed time: {end - start:.4f} seconds")

<h3 style="color:rgb(37, 131, 207);">Evaluating the basic model</h3>

In [ ]:
# Using the best of the basic models
model_basic = tf.keras.models.load_model('best_model.keras')

In [ ]:
# Evaluate the basic model on the test dataset
test_loss_basic, test_accuracy_basic = model_basic.evaluate(test_dataset)

In [ ]:
# Load training history
with open('training_history.json', 'r') as file:
    training_history = json.load(file)

In [ ]:
training_history["accuracy"]

In [ ]:
training_history["val_accuracy"]

In [ ]:
training_history["loss"]

In [ ]:
training_history["val_loss"]

In [ ]:
# plot the data
#settings
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
epochs = np.arange(len(training_history["accuracy"]))
fs = 14
fs_ticks = 10
# training and validation accuracy
ax[0].plot(epochs, training_history["accuracy"], label="Training accuracy")
ax[0].plot(epochs, training_history["val_accuracy"], label="Validation accuracy")
ax[0].set_ylabel("Accuracy", fontsize=fs)
ax[0].grid(True, alpha=0.5)
ax[0].set_xlabel("Epochs", fontsize=fs)
ax[0].legend(frameon=False, fontsize=fs)

# trainng and validation loss
ax[1].plot(epochs, training_history["loss"], label="Training loss")
ax[1].plot(epochs, training_history["val_loss"], label="Validation loss")
ax[1].set_ylabel("Loss", fontsize=fs)
ax[1].grid(True, alpha=0.5)
ax[1].set_xlabel("Epochs", fontsize=fs)
ax[1].legend(frameon=False, fontsize=fs)

plt.tight_layout()
plt.savefig("../plot/accuracy_loss_basic.png", dpi=600, bbox_inches="tight")
plt.show()

In [ ]:
type(test_dataset)

In [ ]:
y_true_basic = []
y_pred_basic = []

for X_batch_basic, labels in test_dataset:
    preds = model_basic.predict(X_batch_basic).flatten()
    bin_preds = (preds > 0.5).astype("int32")
    y_true_basic.extend(labels.numpy())
    y_pred_basic.extend(bin_preds)

In [ ]:
y_true_basic

In [ ]:
y_pred_basic

In [ ]:
#ground_truth is your y_true
#prediction is your y_pred
ground_truth_basic = y_true_basic
predictions_basic = y_pred_basic

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
class_names

In [ ]:
# Compute confusion a
conf_matrix = confusion_matrix(ground_truth_basic, predictions_basic)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.savefig("../plot/cm_basic.png", dpi=600, bbox_inches="tight")
plt.show()

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# Print classification report
print("Classification Report:")
print(classification_report(ground_truth_basic, predictions_basic, target_names=class_names))


In [ ]:
# Get a batch of test data
image_batch, label_batch = test_dataset.as_numpy_iterator().next()

# Generate predictions
predictions_basic = model_basic.predict_on_batch(image_batch).flatten()
predictions_basic = tf.where(predictions_basic < 0.5, 0, 1).numpy()  # Apply threshold for binary classification

# Visualize predictions
plt.figure(figsize=(16, 6))
for i in range(16):  # Increase the number of images to 16
    ax = plt.subplot(2, 8, i + 1)  # Adjust the grid to 4x4
    plt.imshow(image_batch[i].astype("uint8"))
    true_label = label_batch[i]
    pred_label = predictions_basic[i]
    title = f"True: {class_names[true_label]}\nPred: {class_names[pred_label]}"
    color = 'green' if true_label == pred_label else 'red'
    plt.title(title, color=color, fontsize=14)  # Make the font size smaller
    plt.axis("off")
plt.suptitle("Predictions on Test Data", fontsize=16, fontweight='bold')
plt.tight_layout()
#plt.subplots_adjust(top=0.92)
plt.savefig("../plot/predictions_basic_2.png", dpi=600, bbox_inches="tight")
plt.show()